In [ ]:
from io import StringIO
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_style("darkgrid")
plt.rcParams.update({'font.size': 20})

In [ ]:
def load_pmt(fname):
    """Load PMT data and markers from a log file."""
    with open(fname) as fh:
        raw = fh.readlines()
    pmt_raw = "".join([line for line in raw if not line.startswith("M")])
    pmt_markers_raw = "".join([line for line in raw if line.startswith("M")])
    pmt_data = np.genfromtxt(StringIO(pmt_raw), names=True, dtype=None)
    pmt_markers = np.genfromtxt(StringIO(pmt_markers_raw), dtype=None, encoding='ASCII')
    return pmt_data, pmt_markers

def load_ps3(fname):
    ps3_data = np.genfromtxt(fname, names=True, dtype=None, encoding='ASCII')
    return ps3_data

In [ ]:
folder = 'ad4000'
ps3_data = load_ps3(f"{folder}/pmt-powersensor3.log")
pmt_data, pmt_markers = load_pmt(f"{folder}/pmt-nvml.log")

In [ ]:
ps3_data_lr = ps3_data[::10]
offset = min(ps3_data["time"]) + 9.3

# Plot setup
fig, ax = plt.subplots(figsize=(15,7))
pmt_color1 = sns.color_palette()[2]
pmt_color2 = sns.color_palette()[5]

# Plot PowerSensor3
x = ps3_data_lr["time"] - offset + 0.1
y = ps3_data_lr["power_total"]
ax.plot(x, y, '-o', markersize=0.5, linewidth=0.1, color='black', label="PowerSensor3")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Power (W)")
ax.set_xlim(0, 5.9)

# Plot PMT
x2 = pmt_data["timestamp"] - offset
y2 = pmt_data["gpu_instant"]
ax.plot(x2, y2, '-o', markersize=3, linewidth=0.5, color=pmt_color1, label="NVML instant")
y3 = pmt_data["gpu_average"]
ax.plot(x2, y3, '-o', markersize=3, linewidth=0.5, color=pmt_color2, label="NVML average")

# Inset plot
axins = ax.inset_axes([0.62, 0.22, 0.35, 0.4])  # [x, y, width, height]
axins.plot(x, y, '-o', markersize=0.5, linewidth=0.1, color='black')
axins.plot(x2, y2, '-o', markersize=3, linewidth=0.5, color=pmt_color1)
axins.plot(x2, y3, '-o', markersize=3, linewidth=0.5, color=pmt_color2)
axins.set_xlim(2.1, 2.55)
axins.set_ylim(102, 128)
axins.set_xticks([])
axins.set_yticks([])

# Add border around the inset
axins.patch.set_edgecolor("black")
axins.patch.set_linestyle('--')
axins.patch.set_linewidth(1.5)

# Add zoom indicator
indicate_lines = ax.indicate_inset_zoom(axins, edgecolor='black', linestyle='--')
for line in indicate_lines[1]:
    line.set_linestyle('--')

# Kernel execution
ax.axvspan(1.05, 2.93, color='gray', alpha=0.3, label="Kernel execution")

# Show plot
fig.tight_layout(pad=1.4)
plt.title("Energy measurement of a synthetic workload on a NVIDIA RTX 4000 Ada GPU")
plt.legend(loc='upper right')
plt.savefig(f"PMT_{folder.upper()}.pdf")
plt.show()

In [ ]:
folder = 'w7700'
ps3_data = load_ps3(f"{folder}/pmt-powersensor3.log")
pmt_data, pmt_markers = load_pmt(f"{folder}/pmt-amdsmi.log")

In [ ]:
ps3_data_lr = ps3_data[::10].copy()
ps3_data_lr['power_total'] *= 0.85
offset = min(ps3_data["time"]) + 16.1

# Plot setup
fig, ax = plt.subplots(figsize=(15,7))
pmt_color = sns.color_palette()[3]

# Plot PowerSensor3
x = ps3_data_lr["time"] - offset
y = ps3_data_lr["power_total"]
ax.plot(x, y, '-o', markersize=0.5, linewidth=0.1, color='black', label="PowerSensor3")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Power (W)")
ax.set_xlim(0, 3.9)

# Plot PMT
x2 = pmt_data["timestamp"] - offset
y2 = pmt_data["device"]
ax.plot(x2, y2, '-o', markersize=3, linewidth=0.5, color=pmt_color, label="AMD SMI")

# Inset plot
axins = ax.inset_axes([0.62, 0.10, 0.35, 0.6])  # [x, y, width, height]
axins.plot(x, y, '-o', markersize=0.5, linewidth=0.1, color='black')
axins.plot(x2, y2, '-o', markersize=3, linewidth=0.5, color=pmt_color)
axins.set_xlim(0.68, 1.2)
axins.set_ylim(110, 170)
axins.set_xticks([])
axins.set_yticks([])

# Add border around the inset
axins.patch.set_edgecolor("black")
axins.patch.set_linestyle('--')
axins.patch.set_linewidth(1.5)

# Add zoom indicator
indicate_lines = ax.indicate_inset_zoom(axins, edgecolor='black', linestyle='--')
for line in indicate_lines[1]:
    line.set_linestyle('--')

# Kernel execution
ax.axvspan(0.70, 2.23, color='gray', alpha=0.3, label="Kernel execution")

# Show plot
fig.tight_layout(pad=1.4)
plt.title("Energy measurement of a synthetic workload on a AMD W7700 GPU")
plt.legend(loc='upper right')
plt.savefig(f"PMT_{folder.upper()}.pdf")
plt.show()